## Machine Learning workshop
### Part 2: Question Answering with a Fine-Tuned BERT (and TTS example)
This notebook is simply an exampla what relatively recent models are capable of. For more info refer to the course materials and corresponding [lecture on Question Answering](https://youtu.be/pGu426el_IQ) (for now in Russian only).

*If running on colab, uncomment the following cell*

In [ ]:
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install squad_ru_rubert

In [ ]:
# Pre-downloading the BERT for Russian language

!wget https://www.dropbox.com/s/7za1o6vaffbdlcg/rubert_cased_L-12_H-768_A-12_v1.tar.gz -nc
!wget https://www.dropbox.com/s/ns8280pd9t9n9dc/squad_model_ru_rubert.tar.gz -nc

!mkdir -p /root/.deeppavlov/downloads/bert_models/
!mkdir -p /root/.deeppavlov/models/

!tar -xzvf rubert_cased_L-12_H-768_A-12_v1.tar.gz -C /root/.deeppavlov/downloads/bert_models
!tar -xzvf squad_model_ru_rubert.tar.gz -C /root/.deeppavlov/models

#### RuBERT for question answering.
Here we will use the model pre-trained on the SberQuAD dataset from the [SDSJ-2017 challenge problem B](https://github.com/sberbank-ai/data-science-journey-2017/tree/master/problem_B).

In [ ]:
from deeppavlov import build_model, configs

In [ ]:
model_ru = build_model(configs.squad.squad_ru_rubert, download=False)

The following text is copied from [habr post on Crew Dragon flight](https://habr.com/ru/news/t/504642/).

In [ ]:
text = """Первая многоразовая ступень ракеты-носителя Falcon 9 успешно отделилась через две с половиной минуты после старта и автоматически приземлилась на плавучую платформу Of Course I Still Love You у берегов Флориды. Через 12 минут после запуска космический корабль Crew Dragon вышел на расчетную орбиту и отделился от второй ступени ракеты.

Сближение корабля Crew Dragon с Международной космической станцией запланировано на 31 мая. К стыковочному адаптеру на узловом модуле «Гармония» американского сегмента МКС Crew Dragon должен причалить в ручном или, при необходимости, в автоматическом режиме. Эта процедура запланирована на 10:29 по времени Восточного побережья США (17:29 по московскому времени).

В испытательном полете DM2 астронавт Херли является командиром космического корабля (spacecraft commander), а его напарник Бенкен — командир по операциям стыковки и расстыковки (joint operations commander). Фактически это означает, что именно Херли управляет Crew Dragon в полете к МКС, к которой они должны пристыковаться в течение суток после старта. Херли и Бенкен также будут выполнять необходимые для сертификации НАСА проверки систем корабля в полете.

Во время полета Херли и Бенкен провели небольшую экскурсию по Crew Dragon."""

# Source: https://habr.com/ru/news/t/504642/

In [ ]:
print(text)

In [ ]:
model_ru([text], ['Когда отделилась первая ступень?'])

In [ ]:
model_ru([text], ['На какую дату запланирована стыковка?'])

In [ ]:
model_ru([text], ['Кто участвует в полете?'])

In [ ]:
model_ru([text], ['Кто участвует в полете кроме астронавта Херли?'])

In [ ]:
model_ru([text], ['Какая ступень приземлилась на плавучую платформу Of Course I Still Love You?'])

#### Extra. Text to speech (with Silero Models).
[Great post](https://habr.com/ru/post/549480/) on TTS (in Russian). This code belongs to this original repository: https://github.com/snakers4/silero-models

In [ ]:
!pip install -q torchaudio omegaconf

In [ ]:
import numpy as np
from scipy.io.wavfile import write

import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display

In [ ]:
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

# see latest avaiable models
available_languages = list(models.tts_models.keys())
print(f'Available languages {available_languages}')

for lang in available_languages:
    speakers = list(models.tts_models.get(lang).keys())
    print(f'Available speakers for {lang}: {speakers}')

In [ ]:
language = 'ru'
speaker = 'kseniya_16khz'
device = torch.device('cpu')
model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                      model='silero_tts',
                                                                      language=language,
                                                                      speaker=speaker)
model = model.to(device)  # gpu or cpu


audio = apply_tts(texts=[example_text],
                  model=model,
                  sample_rate=sample_rate,
                  symbols=symbols,
                  device=device)

print(example_text)
display(Audio(audio[0], rate=sample_rate))

In [ ]:
audio = apply_tts(texts=["Дерзайте знать! Спасибо за внимание!"],
                  model=model,
                  sample_rate=sample_rate,
                  symbols=symbols,
                  device=device)

display(Audio(audio[0], rate=sample_rate))